In [1]:
#imports
import datetime
import pandas as pd
import plotly.graph_objects as go #importing Plotly's graph objects that we'll use later; read more here: https://plotly.com/python/graph-objects/

/Users/yingzhao/.pyenv/versions/3.8.10/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [11]:
#defining how my text fonts will look like to use it later
TEXTFONT = dict(family="Roboto", size=16, color="#114B5F")

#reading a .csv which is on GitHub
schools_data = pd.read_csv('https://raw.githubusercontent.com/iMEdD-Lab/open-data/master/COVID-19/schools.csv')

#defining my mapbox token that I'll need later
#read more about mapbox access tokens here: 
#https://docs.mapbox.com/help/getting-started/access-tokens/ and
#https://docs.mapbox.com/help/getting-started/access-tokens/
#and you can get your mapbox access token; paste it inside the quotation marks to store it as a string variable
#mind the usage cost!
mapbox_token = 'pk.eyJ1IjoieWluZ3poYW8iLCJhIjoiY2tyd2p2am90MGhkejJ1bjA1c29zb2YyciJ9.2EyBahJUJZocGKuQmQ5ZKA'

#defining date/time stamp the Greek way and then the English way; I'll use it later to make an annotation with my last update
now_gr = pd.datetime.now().strftime('%d/%m/%Y, %H:%M')
now_en = pd.datetime.now().strftime('%m/%d/%Y, %H:%M')

<ipython-input-11-293e7a7d6e7e>:16: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  now_gr = pd.datetime.now().strftime('%d/%m/%Y, %H:%M')
<ipython-input-11-293e7a7d6e7e>:17: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  now_en = pd.datetime.now().strftime('%m/%d/%Y, %H:%M')


In [10]:
#initializing a function
def schools_map_greece(name, schools_data, mapbox_token, show=True, lang="EL"):
    
    #I'll call my schools_data, schools from this point onwards
    schools = schools_data
    
    #I'm doing some cleaning that you don't care about: basicall, I' m making a list with duplicated entries of schools
    REPEATED = list(schools['school'].value_counts()[schools["school"].value_counts() > 1].index)

    
    #Keeping up the cleaning work here: I iterate through my dataset and I'm dropping duplicates for schools 
    #that are in the REPEATED list; I'm keeping the first, 
    #because I know that the first row is always the most updated amongst duplicates
    for i, item in schools.iterrows():
        if item['school'] in REPEATED:
            schools = schools.drop_duplicates(subset=['school'],keep = 'first')
    
    #I'm creating two variables that I'm gonna use later
    
    #this is the sum of schools that they suspended their operations 
    oliki_val = str(schools[schools.status=='Ολική αναστολή'].school.value_counts().sum())
#   #this is the sum of schools under partial suspension (only specific classes suspended their operations)
    meriki_val = str(schools[schools.status=='Μερική αναστολή'].school.value_counts().sum())
    
    #I'm filtering out my main dataframe based on status and I'm creating two different smaller dataframes
    
    #this is the one with schools that they suspended their operations 
    oliki_df = schools[schools.status=='Ολική αναστολή']
    #this is the one with schools under partial suspension (only specific classes suspended their operations)
    meriki_df = schools[schools.status=='Μερική αναστολή']


    #I'm creating my first Scattermapbox here for schools totally closed
    oliki = go.Scattermapbox(
                            #defining latitute, based on relevant df column
                            lat=oliki_df['lat'],
                            #defining longitude, based on relevant df column
                            lon=oliki_df['lng'],
                            #defing mode of my Scattermapbox which is "markers", because
                            #I want this to look like a points map
                            mode='markers',
                            #defing text in tooltip when hovering
                            text=oliki_df['school'] + '<br><i>έως και ' + oliki_df['closed_to'] + '</i>',
                            #defining legend value (Greek and English version)
                            name = 'Ολική αναστολή' if lang=='EL' else 'Schools suspended' + " (" + oliki_val + ")",
                            #I want only text to be included in my tooltip when hovering
                            hovertemplate= '%{text} <extra></extra>',
                            #I want my tooltip to appear below the point I'm hovering over
                            below='', 
                            #defing markers' style (=how the points will look like)
                            marker=dict(size=10, color ='#BA3A0B',opacity=.9),
                            )

    #I'm creating my second Scattermapbox here for schools partially closed
    meriki = go.Scattermapbox(lat=meriki_df['lat'],
            lon=meriki_df['lng'],
            mode='markers',
            text=meriki_df['school'] + '<br>' + meriki_df['label_info'] + '<br><i>έως και ' + meriki_df['closed_to'] + '</i>',
            name = 'Μερική αναστολή' if lang=='EL' else 'Schools suspended' + " (" + meriki_val + ")",
            hovertemplate= '%{text} <extra></extra>',
            below='', 
            marker=dict(size=10, color ='#338C83', opacity=.9)) 
    
    #I'm creating the desired layout of the graphci
    layout = go.Layout(
                    #giving it a title for both the Greek and the English version
                    title=dict(text= '<b>Σχολεία</b> σε <b>αναστολή</b> λειτουργίας στην Ελλάδα' if lang=='EL'\
                                  else '<b>Suspension</b> of operations in <b>schools</b> in Greece', font=TEXTFONT), 
                    #where I want the title to be located in my graphic
                    title_x = 0.02,
                    #desired dimensions of the graphic
                    width=600,
                    height=600,
                    #I want the legend to be displayed
                    showlegend=True,
                    #I'm customizing the legend
                    legend=dict(font=dict(
                                         family="Roboto", 
                                         size=10, color="#114B5F"),
                               orientation='h',
                               yanchor='top', x=0.025
                               ),
                    #I want keys to be displayed in reverse order
                    legend_traceorder="reversed",
                    #Defining background colors
                    paper_bgcolor="#F8FAFA",
                    plot_bgcolor="#F8FAFA",
                    #Defining fonts for my text in hover
                    hoverlabel=dict(font_size=10, font_family="Roboto"),
                    #I want my tooltip to be as close to the point I hover over as it gets
                    hovermode="closest",
                    #Defining the margins of the graphic
                    margin=dict(l=10, r=10, t=70, b=10),
                    #I don't want a title for my legend, but if I wanted, I would need to customize it
                    legend_title=dict(text="", font=dict(family="Roboto", size=12, color="#114B5F")),
                    #Defining how my mapbox will look like
                    mapbox = dict(
                                 #what its center gonna be
                                 center= dict(lat=37.983810, lon=23.727539),    
                                 #what is the access token
                                 accesstoken= mapbox_token,
                                 #how much I want to zoom in 
                                 zoom=4.8,
                                 style="light"                                
                               )
                      )
    
    #initializing my figure (=my graphic and setting the order of the overalying Scattermapboxes)
    fig=go.Figure(data=[meriki,oliki], layout=layout)
    
    #adding annotations
    fig.add_annotation(
                        xref="paper",yref="paper",x=0.01, y=1.07,
                        text='<i>Τελευταία ενημέρωση:</i>'+f"<i> {now_gr} </i> | <a href='https://bit.ly/2TT97VC'>lab.imedd.org/covid19</a>" if lang=="EL"\
                                   else '<i>Last update:</i>'+f"<i> {now_en} </i> | <a href='https://bit.ly/3oZaJeG'>lab.imedd.org/covid19</a>",
                        font=dict(family='Roboto',size=8,color='#114B5F'),
                        showarrow=False,
                        xanchor="left", yanchor="top"
                      )
    
    fig.add_annotation(
                        xref="paper",yref="paper", x=1, y= -0.05,
                        text="<i>Πηγή δεδομένων: <a href='https://bit.ly/3dDwTQn'>Yπ. Παιδείας</a>" if lang=="EL" else "<i>Data Source: <a href='https://bit.ly/2JG3OqP'>Min. of Education</a>",
                        font=dict(family='Roboto',size=8,color='#114B5F'),
                        showarrow=False,
                        xanchor="right", yanchor="bottom"
                      )
    
    fig.add_annotation(
                        xref="paper",yref="paper", x=1, y= -0.05,
                        text="<i>Δεν περιλαμβάνονται σχολεία που είναι κλειστά, στο πλαίσιο μέτρων σε ισχύ.</i>" if lang=="EL"\
                                           else "<i>Schools which are closed as part of measures in force are not included.</i>",
                        font=dict(family='Roboto',size=8,color='#114B5F'),
                        showarrow=False,
                        xanchor="right", yanchor="top"
                      )
    
    #setting configurations
    if show:
        config = dict(
            {
                #I don't want Plotly's toolbar to be displayed
                "displayModeBar": False,
                "scrollZoom": True,
                "displaylogo": False,
                "responsive": True,
                "staticPlot": False,
            }
        )
        
        fig.show(config=config)

In [12]:
#Calling the function I wrote to create the map
#my parameters inside the parenthesis are: name of the graphic, my data, my mapbox access token, show option, desired version
schools_map_greece(name='schools_map_greece', schools_data, mapbox_token, show=True, lang="EN")

SyntaxError: positional argument follows keyword argument (<ipython-input-12-269c291a76ad>, line 3)